In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import tensorflow as tf
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from importlib import reload

# custom module for capstone 2
import cap2tools as c2t
reload(c2t)

Using TensorFlow backend.


<module 'cap2tools' from 'C:\\Users\\Nils\\Documents\\GitHub\\Springboard-Capstone-2-local-yelp\\cap2tools.py'>

In [2]:
# configure GPU memory usage by tensorflow
config = K.tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.90
K.tensorflow_backend.set_session(K.tf.Session(config=config))

In [3]:
# define paths to image directories
train_path = 'downsampled/train'
valid_path = 'downsampled/val'

# create data generators
train_batches, valid_batches = c2t.build_datagens(train_path, valid_path, augment=True)

Found 5480 images belonging to 5 classes.
Found 525 images belonging to 5 classes.


In [4]:
# test different conditions for model
width = 1200
lr = 0.0001
d1s = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
replicates = 3
n_epochs = 10

model_paths = dict()
histories = dict()

for d1 in d1s:
    condition = 'dropout1_{}'.format(str(d1).replace('.', '-'))
    histories[d1] = c2t.run_in_replicate(width, condition, train_batches, valid_batches, 
                                         replicates=replicates, n_epochs=n_epochs, new_weights=False, 
                                         trainable=True, learning_rate=lr, dropout1=d1)
    
    # add key-value pairs to model_paths
    for i in range(1, replicates+1):
        key = condition + ' - ' + str(i)
        value = 'models/vgg16_{}_{}.h5'.format(condition, str(i))
        model_paths[key] = value

2018-09-18 10:34:41 - Started training models/vgg16_dropout1_0_1
2018-09-18 11:01:28 - Started training models/vgg16_dropout1_0_2
2018-09-18 11:28:00 - Started training models/vgg16_dropout1_0_3
2018-09-18 11:54:41 - Started training models/vgg16_dropout1_0-1_1
2018-09-18 12:21:21 - Started training models/vgg16_dropout1_0-1_2
2018-09-18 12:48:12 - Started training models/vgg16_dropout1_0-1_3
2018-09-18 13:14:58 - Started training models/vgg16_dropout1_0-2_1
2018-09-18 13:41:48 - Started training models/vgg16_dropout1_0-2_2
2018-09-18 14:08:27 - Started training models/vgg16_dropout1_0-2_3
2018-09-18 14:35:06 - Started training models/vgg16_dropout1_0-3_1
2018-09-18 15:01:43 - Started training models/vgg16_dropout1_0-3_2
2018-09-18 15:28:25 - Started training models/vgg16_dropout1_0-3_3


RuntimeError: Can't decrement id ref count (unable to extend file properly)

In [ ]:
# evaluate trained models on validation dataset
model_metrics = c2t.eval_models(model_paths, data_path)

In [ ]:
# create table of evaluation results
table = c2t.eval_table(model_metrics, 'Dropout1')
table

In [ ]:
# save history as json
pd.DataFrame(histories).transpose().to_json('VGG16_dropout1_comparison_history.json')

In [ ]:
hist_df = pd.read_json('VGG16_dropout1_comparison_history.json', convert_axes=False)

# plot model comparison
c2t.plot_metric('acc', hist_df)
c2t.plot_metric('loss', hist_df)
c2t.plot_metric('val_acc', hist_df)
c2t.plot_metric('val_loss', hist_df)